In [12]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Crawl Data

In [13]:
def covertCompetition(comp):
    url = 'https://fbref.com/en/'
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    competition = soup.select('span')
    competition = [c.get("href") for c in soup.find_all('a')]
    competition = [c for c in competition if c and 'en/comps/' in c]
    competition_url = ''
    for i in competition:
        # Đếm số ký tự '/'
        count = i.count('/')
        # Lấy chữ cuối cùng sau '/'
        result = i.split('/')[-1] 
        if count == 4 and comp == result:
            competition_url = i
    return competition_url

In [14]:
# https://fbref.com/en/comps/9/Premier-League-Stats
# https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats
def crawl_data(year,comp):
    all_matches = []
    comp = covertCompetition(comp)
    standings_url = f"https://fbref.com{comp}"
    comp = comp.split('/')[-1]
    competitions_name = comp.replace("-Stats", "")
    competitions_name = competitions_name.replace("-"," ")

    previous_season = f"{year}-{year+1}"
    standings_url = standings_url.replace(comp,f"{previous_season}/{previous_season}-{comp}")

    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        soup = BeautifulSoup(data.text)
        # Scores & Fixtures Table
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        # Shotting Table
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]    
        shooting.columns = shooting.columns.droplevel()  
        # Kiểm tra nếu năm <= 2017 thì thêm cột FK
        if(year<=2017):
            shooting['FK'] = ''   
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == str(competitions_name)] 
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        break
        time.sleep(15)  
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    return match_df

In [15]:
df = crawl_data(2021,'Premier-League-Stats')
df.to_csv(f'test.csv', index=False)

In [16]:
test_data = pd.read_csv("test.csv")
test_data.head(10)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18,4,17.3,1.0,0,0,2021,Manchester City
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16,4,18.5,1.0,0,0,2021,Manchester City
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25,10,14.8,0.0,0,0,2021,Manchester City
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25,8,14.3,0.0,0,0,2021,Manchester City
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16,1,16.4,1.0,0,0,2021,Manchester City
5,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,...,Match Report,NaN,15,3,17.1,0.0,0,0,2021,Manchester City
6,2021-10-03,16:30,Premier League,Matchweek 7,Sun,Away,D,2,2,Liverpool,...,Match Report,NaN,12,3,17.0,1.0,0,0,2021,Manchester City
7,2021-10-16,15:00,Premier League,Matchweek 8,Sat,Home,W,2,0,Burnley,...,Match Report,NaN,15,6,14.4,0.0,0,0,2021,Manchester City
8,2021-10-23,17:30,Premier League,Matchweek 9,Sat,Away,W,4,1,Brighton,...,Match Report,NaN,23,12,16.0,1.0,0,0,2021,Manchester City
9,2021-10-30,15:00,Premier League,Matchweek 10,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,14,3,17.7,0.0,0,0,2021,Manchester City
